In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [2]:
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
)
test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor(),
)

Using downloaded and verified file: data/FashionMNIST/raw/train-images-idx3-ubyte.gz
Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw

Using downloaded and verified file: data/FashionMNIST/raw/train-labels-idx1-ubyte.gz
Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Processing...


/home/aditya/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1616554796012/work/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [6]:
batch_size = 64
#Create data loaders
train_dataloader = DataLoader(training_data,batch_size = batch_size)
test_dataloader = DataLoader(test_data,batch_size = batch_size)
for X,y in test_dataloader:
    print("Shape pf X[N,C,H,W]: ",X.shape)
    print("Shape of y:", y.shape,y.dtype)
    break;

Shape pf X[N,C,H,W]:  torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [10]:
device = "cpu"
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
            nn.ReLU()
        )
    def forward(self,x):
        x = self.flatten(x);
        logits = self.linear_relu_stack(x);
        return logits
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [22]:
X = torch.rand(1,28,28, device = device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class:{y_pred}")

Predicted class:tensor([2])


In [29]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [30]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [34]:
layer1 = nn.Linear(in_features = 28*28,out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [35]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.3391, -0.4071,  0.4259,  0.1217,  0.0663, -0.1794,  0.5269,  0.1116,
         -0.0503, -0.0156, -0.8632,  0.1132, -0.1499, -0.1554,  0.3423,  0.1797,
          0.2342,  0.0793,  0.7147,  0.4558],
        [ 0.1152,  0.0057,  0.3075,  0.2161, -0.0940,  0.1436,  0.2408,  0.0049,
         -0.4983, -0.1316, -0.2202,  0.3848,  0.0146, -0.0682,  0.2400,  0.7193,
          0.3585,  0.3048,  0.8669,  0.2543],
        [-0.1006, -0.3820,  0.2890,  0.0320,  0.1251, -0.1713,  0.0840,  0.1294,
         -0.6964,  0.0189, -0.1214,  0.5666,  0.1528,  0.1654, -0.2570,  0.2260,
          0.3645,  0.0357,  0.7629,  0.0299]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0000, 0.0000, 0.4259, 0.1217, 0.0663, 0.0000, 0.5269, 0.1116, 0.0000,
         0.0000, 0.0000, 0.1132, 0.0000, 0.0000, 0.3423, 0.1797, 0.2342, 0.0793,
         0.7147, 0.4558],
        [0.1152, 0.0057, 0.3075, 0.2161, 0.0000, 0.1436, 0.2408, 0.0049, 0.0000,
         0.0000, 0.0000, 0.3848, 0.0146, 0.0000, 0.240

In [36]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [38]:
softmax = nn.Softmax(dim = 0)
pred_probab = softmax(logits)

In [40]:
print("Model structure: ",model,"\n\n")
for name,param in model.named_parameters():
    print(f"Layer:{name}| Size:{param.size()}|Values:{param[:2]}")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer:linear_relu_stack.0.weight| Size:torch.Size([512, 784])|Values:tensor([[ 0.0122,  0.0176, -0.0206,  ...,  0.0254,  0.0306, -0.0310],
        [ 0.0144,  0.0207,  0.0338,  ..., -0.0260, -0.0035,  0.0085]],
       grad_fn=<SliceBackward>)
Layer:linear_relu_stack.0.bias| Size:torch.Size([512])|Values:tensor([0.0140, 0.0328], grad_fn=<SliceBackward>)
Layer:linear_relu_stack.2.weight| Size:torch.Size([512, 512])|Values:tensor([[ 0.0209, -0.0375, -0.0248,  ..., -0.0363,  0.0197,  0.0061],
        [-0.0070, -0.0184, -0.0092,  ..., -0.0308, -0.0240,  0.0119]],
       grad_fn=<SliceBackward>)
Layer:linear_relu_stack.2.bias| Size:torch.Size(